In [1]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe solutions
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Initialize models
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)

# Open webcam
cap = cv2.VideoCapture(0)
prev_frame_time = 0

def estimate_engagement(face_landmarks, pose_landmarks):
    if not face_landmarks or not pose_landmarks:
        return "No Face/Pose Detected"

    # Get landmark positions
    nose = face_landmarks.landmark[1]
    left_shoulder = pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

    # Heuristics
    head_tilt = abs(left_shoulder.z - right_shoulder.z)
    shoulder_distance = abs(left_shoulder.x - right_shoulder.x)

    if head_tilt > 0.1:
        return "Appears Distracted"
    elif shoulder_distance < 0.2:
        return "Slouched - Low Engagement"
    else:
        return "Highly Engaged"

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty frame.")
        continue

    # Flip and convert color
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process with MediaPipe
    pose_results = pose.process(rgb_frame)
    face_results = face_mesh.process(rgb_frame)

    # Draw Pose
    if pose_results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Draw Face Mesh
    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

    # Estimate engagement
    engagement_status = estimate_engagement(
        face_results.multi_face_landmarks[0] if face_results.multi_face_landmarks else None,
        pose_results.pose_landmarks
    )

    # Overlay status
    cv2.putText(frame, f'Status: {engagement_status}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # FPS calculation
    new_frame_time = time.time()
    fps = int(1 / (new_frame_time - prev_frame_time)) if prev_frame_time else 0
    prev_frame_time = new_frame_time
    cv2.putText(frame, f'FPS: {fps}', (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Body Language Interpreter', frame)

    # Exit on pressing ESC
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 